# Treadmill experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
project = "colasa_3d-vision_revisions"

In [ ]:
import flexiznam as flz

flm_sess = flz.get_flexilims_session(project_id=project)

mice = flz.get_entities(datatype='mouse', flexilims_session=flm_sess)
print(f"{len(mice)} mice in {project}")

In [ ]:
protocol_base = 'SpheresTubeMotor'

treadmill_sessions = {}
all_sessions = flz.get_entities(datatype='session', flexilims_session=flm_sess)
for mouse_name, mouse_data in mice.iterrows():
    sessions = all_sessions[all_sessions.origin_id==mouse_data.id]
    for session_name, sess_data in sessions.iterrows():
        recordings = flz.get_children(parent_id=sess_data.id, flexilims_session=flm_sess, children_datatype='recording')
        if not len(recordings):
            print(f"No recordings for session {session_name}")
            continue
        if protocol_base in recordings.protocol.values:
            treadmill_sessions[session_name] = recordings
print(f"{len(treadmill_sessions)} sessions with treadmill data")

In [ ]:
protocol_base = 'SpheresTubeMotor'
protocol_base = "SpheresPermTubeReward"
SESSION ='PZAG17.3a_S20250402'
for session_name, recordings in treadmill_sessions.items():
    # Get the recordings that are  motor
    if session_name != SESSION:
        continue
    recordings = recordings[recordings.protocol != protocol_base]
    print(f"{len(recordings)} sphere protocols")
    print(recordings.iloc[0].name)
    break




In [ ]:
import flexiznam as flz
from cottage_analysis.analysis.spheres import *
import flexiznam as flz


exclude_datasets = None

harp_is_in_recording = True
use_onix = False
conflicts = "skip"
sync_kwargs = None
ephys_kwargs = None
# We can just run the same pipeline. It will skip depth and rsof fit and just run the
# the rf fit
protocol_base = "SpheresPermTubeReward"
flexilims_session = flz.get_flexilims_session(project_id=project)
assert flexilims_session is not None or project is not None


In [ ]:
from cottage_analysis.analysis import treadmill

filter_datasets = {"anatomical_only": 3}
recording_type = "two_photon"
photodiode_protocol = 5
return_volumes = True


load_onix = False if recording_type == "two_photon" else True
all_imaging_df = []
for i, recording_name in enumerate(sorted(recordings.name)):
    print(f"Processing recording {i+1}/{len(recordings)}")
    recording, harp_recording, onix_rec = get_relevant_recordings(
        recording_name, flexilims_session, harp_is_in_recording, load_onix
    )        
    vs_df = synchronisation.generate_vs_df(
        recording=recording,
        photodiode_protocol=photodiode_protocol,
        flexilims_session=flexilims_session,
        harp_recording=harp_recording,
        onix_recording=onix_rec if use_onix else None,
        project=project,
        conflicts=conflicts,
        sync_kwargs=sync_kwargs,
        protocol_base=protocol_base,
    )
    imaging_df = synchronisation.generate_imaging_df(
        vs_df=vs_df,
        recording=recording,
        flexilims_session=flexilims_session,
        filter_datasets=filter_datasets,
        exclude_datasets=exclude_datasets,
        return_volumes=return_volumes,
    )
    imaging_df = format_imaging_df(imaging_df=imaging_df, recording=recording)
    imaging_df = treadmill.process_imaging_df(imaging_df, trial_duration=2)
    all_imaging_df.append(imaging_df)

In [ ]:
imaging_df = pd.concat(all_imaging_df, ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt

starts = imaging_df.query('is_trial_start')
ends = imaging_df.query('is_trial_end')

plt.figure(figsize=(7, 5))
t0 = imaging_df.imaging_harptime.min()
for i in range(2):
    plt.subplot(2,1,1+i)
    plt.plot(imaging_df.imaging_harptime-t0, imaging_df.MotorSpeed, label='Motor')
    plt.plot(imaging_df.imaging_harptime-t0, imaging_df.RS*100, label='Actual')
    plt.ylabel('Speed (cm/s)')
    plt.ylim(-5, 70)
    plt.scatter(ends.imaging_harptime-t0, ends.MotorSpeed, color='k')
    plt.scatter(starts.imaging_harptime-t0, starts.MotorSpeed, color='g')
plt.xlim(100, 255)
plt.xlabel('Time (s)')
plt.axvline(163, color='k', zorder=-10)
plt.axvline(165, color='k', zorder=-10)
plt.legend(loc='upper right')
plt.tight_layout()

In [ ]:
from cottage_analysis.analysis import spheres
from cottage_analysis.analysis import treadmill
vs_df, trials_df = treadmill.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    photodiode_protocol=5,
    return_volumes=True,
)


In [ ]:
trials_df.shape

In [ ]:
trials_df['MotorSpeed'] = np.round(treadmill.sps2speed(trials_df['MotorSps_stim'].apply(np.nanmedian).values))
trials_df['expected_optic_flow'] = np.round(trials_df['expected_optic_flow_stim'].apply(np.nanmedian).values)

trials_df.groupby(['MotorSpeed', 'expected_optic_flow']).aggregate(len)['trial_no'].unstack()


In [ ]:
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
suite2p_ds = list(suite2p_ds.values())[0][-1]
fs = suite2p_ds.extra_attributes["fs"]


In [ ]:
suite2p_ds.path_full

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

fig_folder = suite2p_ds.path_full.parent.parent / 'motor_analysis'
fig_folder.mkdir(exist_ok=True)

motor_speeds = 2**np.arange(2,7)
optic_flows =  4**np.arange(6)
all_dffs = np.vstack(trials_df.dff_stim.values)
total_mean = np.nanmean(all_dffs, axis=0)
total_std = np.nanstd(all_dffs, axis=0)

roi = 514

fig, axes = plt.subplots(len(optic_flows), len(motor_speeds), figsize=(10, 10))
nrois = trials_df.dff_stim.iloc[0].shape[1]
for roi in tqdm(range(nrois)):
    ymin = 0
    ymax = 0.05

    for x in axes.flatten():
        x.clear()
    for (motor, optic_flow), df in trials_df.groupby(['MotorSpeed', 'expected_optic_flow']):
        dffs = df.dff_stim.values
        shapes = np.vstack([dff.shape for dff in dffs])
        m = np.min(shapes[:,0])
        dffs = np.stack([dff[:m,:] for dff in dffs])
        avg = np.nanmean(dffs, axis=0)
        std = np.nanstd(dffs, axis=0)

        m_index = list(motor_speeds).index(motor)
        of_index = list(optic_flows).index(optic_flow)
        ax = axes[len(optic_flows)-of_index-1, m_index]
        #ax.set_title(f"M: {motor}, OF: {optic_flow}")
        time = np.arange(avg.shape[0]) / fs
        # ax.fill_between(time, avg[:,roi]-std[:,roi], avg[:,roi]+std[:,roi], alpha=0.5, color='k')
        ax.plot(time, dffs[...,roi].T, color='k', lw=1, alpha=0.7)
        ax.plot(time, avg[:,roi], color='darkorchid', lw=2)
        ax.set_xlim(time.min(), time.max())
        ymin = min(ymin, dffs[...,roi].min())
        ymax = max(ymax, dffs[...,roi].max())

    for x in axes.flatten():
        x.set_ylim(ymin, ymax*1.05)
    for i, m in enumerate(motor_speeds):
        axes[0, i].set_title(f'{int(m)} cm/s')
        axes[-1, i].set_xlabel('Time (s)')
    for i, m in enumerate(optic_flows):
        axes[len(optic_flows)-i-1, 0].set_ylabel(f'OF: {int(m)} deg/s \n dF/F')
    fig.suptitle(f"{session_name} roi {roi}")
    fig.tight_layout()
    fig.savefig(fig_folder / f'psth_roi{roi}.png')

In [ ]:
4*5

In [ ]:
stim_df = imaging_df.query('is_stim')
n_frames = stim_df.groupby(['MotorSpeed', 'expected_optic_flow']).dffs.aggregate(len)
avg = stim_df.groupby(['MotorSpeed', 'expected_optic_flow']).aggregate('mean')

motor_speeds = 2**np.arange(2,7)
optic_flows =  4**np.arange(6)

def compute_motor_of_matrix(df, motor_speeds=motor_speeds, optic_flows=optic_flows):
    avg = df.groupby(['MotorSpeed', 'expected_optic_flow']).aggregate('mean')
    data = avg.dffs.unstack()
    n_neurons = df.dffs.iloc[0].shape[1]
    output = np.zeros([len(motor_speeds), len(optic_flows), n_neurons])
    for ispeed, speed in enumerate(motor_speeds):
        for iof, optic_flow in enumerate(optic_flows):
            output[ispeed,iof, :] = data.loc[speed, optic_flow][0,:]
    return output

output = compute_motor_of_matrix(stim_df)

In [ ]:
# Compute global mean and std for z-scoring
all_dffs = np.dstack(imaging_df.dffs)[0]
total_mean = np.nanmean(all_dffs, axis=1)
total_std = np.nanstd(all_dffs, axis=1)
zscores_max =  (np.max(output, axis=(0,1))- total_mean) / total_std
order = np.argsort(zscores_max)[::-1]


In [ ]:
roi = 514

for (motor, of), df in stim_df.groupby(['MotorSpeed', 'expected_optic_flow']):
    dff = np.dstack(df.dffs.values)
dff.shape


In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

min_motor=0
range_motor = np.diff(np.log2(motor_speeds)[[0,-1]])[0]
range_of = np.diff(np.log2(optic_flows)[[0,-1]])[0]
min_of = 0
print(f'Motor range: {range_motor}, optic flow range: {range_of}')
fig, ax = plt.subplots(1,1)

ax_cb = make_axes_locatable(ax)
# Add an Axes to the right of the main Axes.
cax1 = ax_cb.append_axes("right", size="7%", pad="2%")

data = output[..., roi]
img = ax.imshow(data.T, origin='lower', vmin=0, vmax=data.max(), cmap='viridis', 
                extent=(min_motor, range_motor, min_of, range_of))
ax.set_xticks(np.linspace(min_motor+0.5, range_motor-0.5, len(motor_speeds)))
ax.set_xticklabels(motor_speeds)
ax.set_yticks(np.linspace(min_of+0.5, range_of-0.5, len(optic_flows)))
ax.set_yticklabels(optic_flows)
cb1 = fig.colorbar(img, cax=cax1)
cb1.set_label('DFF')
ax.set_xlabel('Motor speed (cm/s)')
ax.set_ylabel('Optic flow (degrees/s)')

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


min_motor=0
range_motor = np.diff(np.log2(motor_speeds)[[0,-1]])[0]
range_of = np.diff(np.log2(optic_flows)[[0,-1]])[0]
min_of = 0
print(f'Motor range: {range_motor}, optic flow range: {range_of}')
fig, ax = plt.subplots(1,1)
ax_cb = make_axes_locatable(ax)
# Add an Axes to the right of the main Axes.
cax1 = ax_cb.append_axes("right", size="7%", pad="2%")


data = (output[..., roi] - total_mean[roi]) / total_std[roi]
img = ax.imshow(data.T, origin='lower', vmin=-data.max(), vmax=data.max(), cmap='RdBu_r', 
                extent=(min_motor, range_motor, min_of, range_of))
ax.set_xticks(np.linspace(min_motor+0.5, range_motor-0.5, len(motor_speeds)))
ax.set_xticklabels(motor_speeds)
ax.set_yticks(np.linspace(min_of+0.5, range_of-0.5, len(optic_flows)))
ax.set_yticklabels(optic_flows)
cb1 = fig.colorbar(img, cax=cax1)
cb1.set_label('Zscored DFF')
ax.set_xlabel('Motor speed (cm/s)')
ax.set_ylabel('Optic flow (degrees/s)')

In [ ]:
aspect_ratio = range_motor/range_of
fig, axes = plt.subplots(25, 25)
fig.set_size_inches(20,20/ aspect_ratio)
do_zscore = True
for iax, ax in enumerate(axes.flatten()):
    roi = order[iax]
    if do_zscore:
        data = (output[..., roi] - total_mean[roi]) / total_std[roi]
        m = max(2, data.max())
        vmin = -m
        vmax = m
        cmap = 'RdBu_r'
    else:
        data = output[..., roi]
        vmax = max(output[..., roi].max(), 0.1)
        vmin=0
        cmap= 'viridis'
    im = ax.imshow(data.T, origin='lower', vmax=vmax, vmin=vmin, cmap=cmap,extent=(min_motor, range_motor, min_of, range_of))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(roi)
plt.subplots_adjust(wspace=0, hspace=0.01)
plt.tight_layout()

In [ ]:
s = vis_stim_ds.path_full / vis_stim_ds.extra_attributes['csv_files']['FrameLog']
frame_df = pd.read_csv(s)
frame_df.head()